In [2]:
import numpy as np

In [3]:
X_train = np.load("train.npy", allow_pickle=True)
# X_test = np.load("test.npy", allow_pickle=True)
y_train = np.load("train-labels.npy", allow_pickle=True)
# y_test = np.load("test-labels.npy", allow_pickle=True)

In [8]:
# example of a one hot encoding
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
# define data
data = asarray([['red'], ['green'], ['blue']])
print(data)
# define one hot encoding
encoder = OneHotEncoder(sparse=False)
# transform data
onehot = encoder.fit_transform(data)
print(onehot)

[['red']
 ['green']
 ['blue']]
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [9]:
onehot

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [249]:
test_x = X_train[:7008] # [:20]
test_y = y_train[:7008] # [:20]

In [251]:
7008 / 4

1752.0

In [252]:
batch_data = np.array(np.split(test_x,1752,axis=0))
batch_labels = np.array(np.split(test_y,1752,axis=0))

In [253]:
import matplotlib.pyplot as plt

In [254]:
batch_data[0].shape

(4, 56, 75, 3)

In [255]:
batch_labels[0][0]

'nv'

In [256]:
list(set(test_y))

['bcc', 'akiec', 'nv', 'bkl', 'df', 'vasc', 'mel']

In [257]:
test_x.shape
test_y.shape

(7008,)

In [258]:
asarray(list(set(test_y))).reshape(-1, 1)

array([['bcc'],
       ['akiec'],
       ['nv'],
       ['bkl'],
       ['df'],
       ['vasc'],
       ['mel']], dtype='<U5')

In [259]:
asarray([['red'], ['green'], ['blue']])

array([['red'],
       ['green'],
       ['blue']], dtype='<U5')

In [260]:
labels_map = asarray(list(set(y_train))).reshape(-1, 1)

In [261]:
labels_map

array([['bcc'],
       ['akiec'],
       ['nv'],
       ['bkl'],
       ['df'],
       ['vasc'],
       ['mel']], dtype='<U5')

In [262]:
encoder = OneHotEncoder(sparse=False)
onehot = encoder.fit_transform(labels_map)
onehot

array([[0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.]])

In [263]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels_map)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LabelEncoder()

In [264]:
le.classes_

array(['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'], dtype='<U5')

In [265]:
le.transform(["akiec", "bcc", "bcc"])

array([0, 1, 1])

In [266]:
label_mapping = {}
for idx, i in enumerate(labels_map.flatten()):
    label_mapping[i] = onehot[idx]

In [267]:
label_mapping

{'bcc': array([0., 1., 0., 0., 0., 0., 0.]),
 'akiec': array([1., 0., 0., 0., 0., 0., 0.]),
 'nv': array([0., 0., 0., 0., 0., 1., 0.]),
 'bkl': array([0., 0., 1., 0., 0., 0., 0.]),
 'df': array([0., 0., 0., 1., 0., 0., 0.]),
 'vasc': array([0., 0., 0., 0., 0., 0., 1.]),
 'mel': array([0., 0., 0., 0., 1., 0., 0.])}

In [268]:
import torch.nn as nn
import torch.nn.functional as F
import torch


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 15 * 11, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # print("x shape", x.shape)
        x = x.view(-1, 16 * 15 * 11)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [269]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [270]:
for idx, i in enumerate(zip(test_x, test_y), 0):
    data, labels = i

In [273]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(zip(batch_data, batch_labels)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = torch.from_numpy(inputs).view(4, 3, 75, 56).float()
        # label_one_hot = np.array([label_mapping[label] for label in labels])
        label_one_hot = np.array(le.transform(labels))
        labels = torch.from_numpy(label_one_hot)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # print(outputs.shape, "\n", outputs)
        # print(labels.shape, "labels", labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.047
[1,   200] loss: 0.051
[1,   300] loss: 0.040
[1,   400] loss: 0.048
[1,   500] loss: 0.052
[1,   600] loss: 0.050
[1,   700] loss: 0.047
[1,   800] loss: 0.046
[1,   900] loss: 0.050
[1,  1000] loss: 0.047
[1,  1100] loss: 0.051
[1,  1200] loss: 0.048
[1,  1300] loss: 0.047
[1,  1400] loss: 0.046
[1,  1500] loss: 0.050
[1,  1600] loss: 0.046
[1,  1700] loss: 0.046
[2,   100] loss: 0.047
[2,   200] loss: 0.050
[2,   300] loss: 0.039
[2,   400] loss: 0.048
[2,   500] loss: 0.052
[2,   600] loss: 0.048
[2,   700] loss: 0.047
[2,   800] loss: 0.046
[2,   900] loss: 0.048
[2,  1000] loss: 0.048
[2,  1100] loss: 0.051
[2,  1200] loss: 0.048
[2,  1300] loss: 0.046
[2,  1400] loss: 0.046
[2,  1500] loss: 0.050
[2,  1600] loss: 0.045
[2,  1700] loss: 0.045
[3,   100] loss: 0.045
[3,   200] loss: 0.049
[3,   300] loss: 0.038
[3,   400] loss: 0.047
[3,   500] loss: 0.051
[3,   600] loss: 0.047
[3,   700] loss: 0.047
[3,   800] loss: 0.045
[3,   900] loss: 0.047
[3,  1000] 

In [278]:
X_test = np.load("test.npy", allow_pickle=True)
# X_test = np.load("test.npy", allow_pickle=True)
y_test = np.load("test-labels.npy", allow_pickle=True)
# y_test = np.load("test-labels.npy", allow_pickle=True)

In [283]:
len(X_test[:3004]) / 4

751.0

In [284]:
test_batch_data = np.array(np.split(X_test[:3004],751,axis=0))
test_batch_labels = np.array(np.split(y_test[:3004],751,axis=0))

In [286]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(zip(batch_data, batch_labels)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = torch.from_numpy(inputs).view(4, 3, 75, 56).float()
        # label_one_hot = np.array([label_mapping[label] for label in labels])
        label_one_hot = np.array(le.transform(labels))
        labels = torch.from_numpy(label_one_hot)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 69 %


In [287]:
test_correct = 0
test_total = 0
with torch.no_grad():
    for i, data in enumerate(zip(test_batch_data, test_batch_labels)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = torch.from_numpy(inputs).view(4, 3, 75, 56).float()
        # label_one_hot = np.array([label_mapping[label] for label in labels])
        label_one_hot = np.array(le.transform(labels))
        labels = torch.from_numpy(label_one_hot)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * test_correct / test_total))

Accuracy of the network on the 10000 test images: 69 %


In [241]:
len(test_x)

7010

In [248]:
7008 / 4

1752.0

In [192]:
[4, 16, 15, 11]

[4, 16, 15, 11]

In [198]:
2640 / 16 / 15

11.0

In [132]:
batch_data[0][0][0][0].astype(np.double)

array([0.87741597, 0.54901961, 0.64705882])

In [166]:
test1 = torch.from_numpy(batch_data).type(torch.float)

In [167]:
test1[0].shape

torch.Size([4, 56, 75, 3])

In [169]:
x = test1[0].view(4, 3, 75, 56)

In [106]:
type(x)

torch.Tensor

In [85]:
batch_labels[0]

array(['nv', 'mel', 'nv', 'nv', 'mel'], dtype=object)

In [89]:
batch_labels[0]

array(['nv', 'mel', 'nv', 'nv', 'mel'], dtype=object)